In [0]:
%pip install langchain
%pip install langchain-community
%pip install langchain-chroma
%pip install langchain-openai
%pip install openai==1.56.2
%pip install azure-identity==1.19.0
%pip install pydantic==2.9.2
%pip install requests
%pip install gitpython
%pip install python-dotenv
%pip install repomix

In [0]:
import os
from openai import AzureOpenAI
from azure.identity import ClientSecretCredential
######## Boilerplate necessary to make things work in OneLab/OpenLab ########
LAB_VARIANT = "OpenLab" # USE "OneLab" IF YOU'RE USING ONELAB
ENVIRONMENT = "prd" # choose ENVIRONMENT as dev, uat or prd based on environment
def get_openai_urls(lab_variant: str):
  """This function is created to return OpenAI URLs for OpenLab/OneLab. We want to use the function with lazy evaluation so that OneLab doesn't affect OpenLab and vice-versa."""
  if lab_variant == "OpenLab":
    secret_scope = f"{lab_variant}-SecretScope"
    return f"https://{dbutils.secrets.get(scope=secret_scope, key='OpenAiHostname')}openoaisdc-completions-apis/"
  elif lab_variant == "OneLab":
    return f"https://apim-1labgen-ap-apizone-{ENVIRONMENT}01.azure-api.net/openaisdc-completions-apis/"
  else: 
    raise Exception("Invalid lab_variant")

client_id = dbutils.secrets.get(scope=f"{LAB_VARIANT}-SecretScope", key="DataServicePrincipalClientId")
client_secret = dbutils.secrets.get(scope=f"{LAB_VARIANT}-SecretScope", key="DataServicePrincipalClientSecret")
credential = ClientSecretCredential(tenant_id="6e93a626-8aca-4dc1-9191-ce291b4b75a1", client_id=client_id, client_secret=client_secret)
access_token = credential.get_token("https://cognitiveservices.azure.com/.default")

os.environ["AZURE_OPENAI_TOKEN"] = access_token.token
os.environ["AZURE_OPENAI_VERSION"] = "2024-10-21"  # https://learn.microsoft.com/en-us/azure/ai-services/openai/reference#api-specs
os.environ["AZURE_OPENAI_BASE_URL"] = get_openai_urls(LAB_VARIANT)
os.environ["USER_AGENT"] = "myagent"

client = AzureOpenAI(
  api_key=os.environ["AZURE_OPENAI_TOKEN"],  
  api_version=os.environ["AZURE_OPENAI_VERSION"],
  azure_endpoint=os.environ["AZURE_OPENAI_BASE_URL"]
)

In [0]:
def get_completion_without_rag(query: str, context: str) -> str:
      """Returns the completion from GPT4 without any augmentation."""
      completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
          {"role": "system", "content": context}, # <-- This is the system message that provides context to the model
          {"role": "user", "content": query}  # <-- This is the user message for which the model will generate a response
        ],
        seed=1
      )
      return completion.choices[0].message.content
  
def get_completion_with_rag(query: str, context: str, extra_content: str) -> str:
    """Runs a RAG pipeline that will augment the context with extra_content if 'rabo' is in the query. Returns the chat completion."""  
    context += f"Wikipedia page about Rabobank that you can reference: {extra_content}"
    completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": context},  # <-- This is the system message that provides context to the model
        {"role": "user", "content": query}  # <-- This is the user message for which the model will generate a response
    ],
    seed=1
    )
    return completion.choices[0].message.content

In [0]:
get_completion_without_rag(
    query="What makes the sky blue?",
    context="You are a pirate poet. Give creative responses"
)

In [0]:
from prompts.prompt_loader import load_prompt_from_file

prompt = load_prompt_from_file("sample_prompt.txt")